# 📓 The GenAI Revolution Cookbook

**Title:** How to Build Multi-Agent AI Systems with CrewAI, Step by Step

**Description:** Design scalable multi-agent AI systems with CrewAI using YAML configs, task delegation, tools, and training—then build a Customer Feedback Analyst.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Here’s a revised version of your draft content, updated to use the “first\-crew” approach from CrewAI’s guide. I’ve refactored your pipeline to match their project layout, crew base class, agent/task decorators, CLI usage. I also adjusted writing style per your humanization rules.

---

Generative AI gets useful when it ships. In this guide, you’ll build a Customer Feedback Analyst using CrewAI following the “first\-crew” pattern. You’ll define a crew via the CLI, set up agents and tasks in YAML, write tools, and run the system end\-to\-end. You’ll produce a real report with sentiment tables and a pie chart. Plus, you’ll learn the architecture and code you can reuse on your own data. For best practices on extracting structured data with LLMs, check out our [structured data extraction pipeline guide](/article/structured-data-extraction-with-llms-how-to-build-a-pipeline-3).

## Why This Approach Works

Multi\-agent systems shine when you need specialized roles working together. A single LLM call can classify sentiment, but it struggles with reading files, computing aggregates, generating charts, assembling reports all at once. When you split responsibilities into focused agents, you gain modularity and clarity. You can also swap models or tools for specific tasks.

CrewAI gives you a clean model for agents, tasks, tools, and crews. You write config in YAML. To coordinate complex workflows you use agent/task decorators, context passing, and built\-in memory. If you want agents that interoperate and support audits, look into the [Model Context Protocol (MCP)](/article/model-context-protocol-mcp-explained-2025-guide-for-builders).

In this tutorial, you build a sequential pipeline. Each task receives context from earlier steps. That keeps logic simple and traceable. You’ll wire a file reader, a sentiment classifier, an aggregator, a chart generator, and a report writer into a single crew defined with CrewAI’s first\-crew pattern.

## What You’ll Build

By the end, you’ll have:

* An **output/report.md** file with sentiment breakdown and insights.
* A **reports/sentiment\_pie.png** chart visualizing the distribution.
* A reusable pipeline you can adapt to other CSV datasets or feedback sources.

You’ll run everything via CLI and script, validate outputs, and understand why each component exists.

## Setup \& Installation

Start by creating a new CrewAI project using the CLI. This scaffolds the structure for you. ([docs.crewai.com](https://docs.crewai.com/en/guides/crews/first-crew))

In [ ]:
crewai create crew feedback_analyst
cd feedback_analyst

That gives you directories like:

In [ ]:
feedback_analyst/
├── .gitignore
├── pyproject.toml
├── README.md
├── .env
└── src/
    └── feedback_analyst/
        ├── __init__.py
        ├── main.py
        ├── crew.py
        ├── tools/
        │   ├── __init__.py
        │   └── custom_tools.py
        └── config/
            ├── agents.yaml
            └── tasks.yaml

Then install dependencies. Pin versions for stability:

In [ ]:
pip install -U crewai crewai-tools python-dotenv pyyaml pandas matplotlib

Set your API keys in `.env`:

In [ ]:
OPENAI_API_KEY=your-openai-key-here
ANTHROPIC_API_KEY=your-anthropic-key-here # optional

Verify them in your code:

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
required = ["OPENAI_API_KEY"]
missing = [k for k in required if not os.getenv(k)]
if missing:
    raise EnvironmentError(f"Missing required environment variables: {', '.join(missing)}")
print("All required API keys found.")

Create directories if needed:

In [ ]:
from pathlib import Path

ROOT = Path.cwd()
DATA_DIR = ROOT / "data"
REPORTS_DIR = ROOT / "reports"

for d in [DATA_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

Generate a sample CSV if you don’t have one:

In [ ]:
csv_path = DATA_DIR / "customer_feedback.csv"
if not csv_path.exists():
    csv_path.write_text(
        "id,text\n"
        "1,I love the new dashboard, clean and fast.\n"
        "2,App crashed twice during checkout. Very frustrating.\n"
        "3,Support was helpful but resolution took too long.\n"
        "4,Works as expected. No issues so far.\n",
        encoding="utf-8",
    )
    print(f"Created example CSV at {csv_path}")

## Step\-by\-Step Implementation

### Define Tools

In `src/feedback_analyst/tools/custom_tools.py`, write tools for reading, aggregating, charting:

In [ ]:
from crewai_tools import FileReadTool, tool
import json
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

def make_file_reader(csv_path: str):
    return FileReadTool(file_path=csv_path, description="Read customer_feedback.csv")

@tool("aggregate_sentiment")
def aggregate_sentiment(jsonl_str: str) -> str:
    lines = [ln.strip() for ln in jsonl_str.strip().split("\n") if ln.strip()]
    records = [json.loads(ln) for ln in lines]
    df = pd.DataFrame(records)
    counts = df["sentiment"].value_counts().to_dict()
    result = {
        "positive": counts.get("positive", 0),
        "neutral": counts.get("neutral", 0),
        "negative": counts.get("negative", 0),
    }
    return json.dumps(result)

@tool("make_sentiment_pie")
def make_sentiment_pie(summary_json: str) -> str:
    data = json.loads(summary_json)
    counts = [
        data.get("positive", 0),
        data.get("neutral", 0),
        data.get("negative", 0),
    ]
    labels = ["Positive", "Neutral", "Negative"]
    colors = ["#2ecc71", "#95a5a6", "#e74c3c"]

    out_path = Path("reports/sentiment_pie.png")
    out_path.parent.mkdir(parents=True, exist_ok=True)

    fig, ax = plt.subplots(figsize=(5,5), dpi=160)
    ax.pie(counts, labels=labels, autopct="%1.1f%%", startangle=140, colors=colors)
    ax.axis("equal")
    plt.title("Sentiment Distribution")
    plt.tight_layout()
    fig.savefig(out_path)
    plt.close(fig)
    return str(out_path.resolve())

### Configure Agents

Edit `src/feedback_analyst/config/agents.yaml`:

```yaml
sentiment_classifier:
  role: Sentiment Classifier
  goal: Label each feedback entry as positive, neutral, or negative
  backstory: You are an expert at understanding customer tone. You read feedback text and assign accurate sentiment.
  llm: openai/gpt-4o
  tools:
    - file_reader
  verbose: true
  allow_delegation: false

aggregator:
  role: Data Aggregator
  goal: Compute sentiment counts from classified feedback
  backstory: You parse structured data and produce accurate numeric summaries
  llm: openai/gpt-4o
  tools:
    - aggregate_sentiment
  verbose: true
  allow_delegation: false

chart_maker:
  role: Chart Generator
  goal: Create pie chart visualizing sentiment distribution
  backstory: You turn data into clear visualizations
  llm: openai/gpt-4o
  tools:
    - make_sentiment_pie
  verbose: true
  allow_delegation: false

report_writer:
  role: Report Writer
  goal: Assemble final Markdown report with insights and visuals
  backstory: You synthesize analysis into clear actionable reports
  llm: openai/gpt-4o
  tools: []
  verbose: true
  allow_delegation: false
```

### Define Tasks

Edit `src/feedback_analyst/config/tasks.yaml`:

```yaml
classify_feedback:
  description: |
    Read customer_feedback.csv using the file_reader tool.
    For each row, classify the sentiment of the "text" field as positive, neutral, or negative.
    Output one JSON object per line (JSONL) with fields: id, text, sentiment.
  agent: sentiment_classifier
  expected_output: JSONL with id, text, sentiment for each entry

aggregate_counts:
  description: |
    Take JSONL output from classify_feedback.
    Use aggregate_sentiment tool to compute counts of positive, neutral, negative sentiments.
    Return a JSON object with keys: positive, neutral, negative.
  agent: aggregator
  expected_output: JSON object with sentiment counts
  context:
    - classify_feedback

generate_chart:
  description: |
    Take the JSON summary from aggregate_counts.
    Use make_sentiment_pie tool to generate a pie chart PNG.
    Return absolute file path to the chart.
  agent: chart_maker
  expected_output: Absolute path to sentiment_pie.png
  context:
    - aggregate_counts

write_report:
  description: |
    Use sentiment counts and chart path from previous tasks.
    Write a Markdown report that includes:
      - Summary of total feedback entries and sentiment breakdown
      - Table with counts of positive, neutral, negative
      - Embedded image: ![Sentiment Distribution](reports/sentiment_pie.png)
      - Insights or recommendations based on sentiment distribution
    Return Markdown content.
  agent: report_writer
  expected_output: Complete Markdown report with table, chart, and insights
  context:
    - aggregate_counts
    - generate_chart
  output_file: output/report.md
```

### Configure the Crew

In `src/feedback_analyst/crew.py` use the first\-crew pattern: agent and task decorators. Inspired by CrewAI’s guide. ([docs.crewai.com](https://docs.crewai.com/en/guides/crews/first-crew))

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.project import CrewBase, agent, task, crew
from typing import List
from feedback_analyst.tools.custom_tools import make_file_reader, aggregate_sentiment, make_sentiment_pie
from pathlib import Path

@CrewBase
class FeedbackAnalystCrew():
    agents: List[Agent]
    tasks: List[Task]

    @agent
    def sentiment_classifier(self) -> Agent:
        cfg = self.agents_config['sentiment_classifier']
        tools = [ make_file_reader(str(Path('data/customer_feedback.csv'))) ]
        return Agent(
            config=cfg,
            verbose=True,
            tools=tools,
        )

    @agent
    def aggregator(self) -> Agent:
        cfg = self.agents_config['aggregator']
        tools = [ aggregate_sentiment ]
        return Agent(
            config=cfg,
            verbose=True,
            tools=tools,
        )

    @agent
    def chart_maker(self) -> Agent:
        cfg = self.agents_config['chart_maker']
        tools = [ make_sentiment_pie ]
        return Agent(
            config=cfg,
            verbose=True,
            tools=tools,
        )

    @agent
    def report_writer(self) -> Agent:
        cfg = self.agents_config['report_writer']
        return Agent(
            config=cfg,
            verbose=True,
            tools=[],
        )

    @task
    def classify_feedback(self) -> Task:
        return Task(
            config=self.tasks_config['classify_feedback'],
        )

    @task
    def aggregate_counts(self) -> Task:
        return Task(
            config=self.tasks_config['aggregate_counts'],
        )

    @task
    def generate_chart(self) -> Task:
        return Task(
            config=self.tasks_config['generate_chart'],
        )

    @task
    def write_report(self) -> Task:
        return Task(
            config=self.tasks_config['write_report'],
            output_file='output/report.md',
        )

    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )

### Main Script

In `src/feedback_analyst/main.py`:

In [ ]:
import os
from feedback_analyst.crew import FeedbackAnalystCrew

def run():
    os.makedirs('output', exist_ok=True)
    result = FeedbackAnalystCrew().crew().kickoff(inputs={})
    print("\n\n=== FINAL REPORT ===\n\n")
    print(result.raw)
    print("\nReport saved to output/report.md")

if __name__ == "__main__":
    run()

## Running, Saving \& Validating Outputs

From your project root, use:

In [ ]:
crewai run

This runs the crew using the YAML configs and crew.py. You’ll see logs for each agent and task. When it finishes:

* Check `output/report.md` for your report content.
* Confirm `reports/sentiment_pie.png` exists.
* If using a notebook or previewer, render the Markdown and display the chart inline.

## Key Design Decisions

* Deterministic aggregation via Python tool prevents hallucinations or wrong counts.
* Sequential execution ensures logical flow: classification before aggregation; chart after counts.
* YAML for agents/tasks keeps config separate from code. You can swap prompts, models, or tools without modifying core logic.
* Tools each do one job. That keeps components testable and reusable.

## Conclusion \& Next Steps

You’ve built a complete Customer Feedback Analyst following CrewAI’s first\-crew structure. You defined agents and tasks in YAML. You implemented tools for reading data, classification, aggregation, and visualization. You wrote a crew class, ran the system, and produced a Markdown report and PNG chart.

Here’s where you can go next:

* Add schema validation for the JSONL outputs using Pydantic to catch malformed data early.
* Implement chunking to handle large CSVs without hitting context limits.
* Integrate observability: log token usage, task durations, and tool calls for cost/performance insights.
* Experiment with parallel or dynamic task execution when steps don’t strictly depend on each other.

Use this template on other datasets. Adapt the agents, tweak the goals, change tools. You now have a strong foundation for multi\-agent pipelines that deliver structured, validated outputs.